In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
if sys.version_info >= (3,8):
    import pickle
else:
    # !pip install pickle5
    import pickle5 as pickle
import os

Mounted at /content/drive


In [ ]:
# load data - data can be found here:
# https://drive.google.com/drive/folders/1QJokFJZh1mc6PwskF3M5aWXcUzgzSQ3K?usp=sharing
os.listdir('/content/drive/My Drive/')

with open('/content/drive/My Drive/Colab_Files/emopia/featuresnp.pickle', 'rb') as f:
    featuresnp = pickle.load( f )

with open('/content/drive/My Drive/Colab_Files/emopia/domiantQsnp.pickle', 'rb') as f:
    dominantQsnp = pickle.load( f )

# outputs need to be in range starting from 0
dominantQsnp = dominantQsnp - 1

print('featuresnp.shape: ', featuresnp.shape)
print('dominantQsnp.shape: ', dominantQsnp.shape)

featuresnp.shape:  (1078, 17)
dominantQsnp.shape:  (1078,)


In [ ]:
# scale features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True, with_std=True)
scaled_features = scaler.fit_transform( featuresnp )

In [ ]:
# split data
from sklearn.model_selection import StratifiedShuffleSplit

stratsplit = StratifiedShuffleSplit( n_splits = 1, test_size = 0.2, random_state = 1)
idxs_generator = stratsplit.split( featuresnp, dominantQsnp )
idxs_list = list (idxs_generator)
train_idxs = idxs_list[0][0]
test_idxs = idxs_list[0][1]

train_input = scaled_features[ train_idxs , : ]
train_output = dominantQsnp[ train_idxs ]

test_input = scaled_features[ test_idxs , : ]
test_output = dominantQsnp[ test_idxs ]

train_input.shape:  (862, 17)


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(17,))) # we need to specify input
model.add(keras.layers.Dense(32, activation='tanh'))
model.add(keras.layers.Dense(4, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 32)                576       
                                                                 
 dense_23 (Dense)            (None, 4)                 132       
                                                                 
Total params: 708
Trainable params: 708
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_input, train_output, batch_size=16, epochs=100) # check out batch size and compare it with training instances

Epoch 1/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9064 - accuracy: 0.6357
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9050 - accuracy: 0.6334
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 0.9043 - accuracy: 0.6392
Epoch 4/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9031 - accuracy: 0.6323
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9020 - accuracy: 0.6415
Epoch 6/100
54/54 [==============================] - 0s 1ms/step - loss: 0.9004 - accuracy: 0.6392
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 0.8998 - accuracy: 0.6415
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 0.8988 - accuracy: 0.6392
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 0.8974 - accuracy: 0.6497
Epoch 10/100
54/54 [==============================] - 0s 1ms/step - loss: 0.8963 - accuracy: 0.6415
Epoch 11/

In [ ]:
model.evaluate( test_input, test_output )

7/7 [==============================] - 0s 2ms/step - loss: 0.9932 - accuracy: 0.5787


[0.9932020902633667, 0.5787037014961243]

In [ ]:
# let's make a beefier model
model1 = keras.models.Sequential()
model1.add(keras.Input(shape=(17,))) # we need to specify input
model1.add(keras.layers.Dense(128, activation='tanh'))
model1.add(keras.layers.Dense(4, activation='softmax'))

model1.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 128)               2304      
                                                                 
 dense_28 (Dense)            (None, 4)                 516       
                                                                 
Total params: 2,820
Trainable params: 2,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history1 = model1.fit(train_input, train_output, epochs=100)

Epoch 1/100
27/27 [==============================] - 0s 1ms/step - loss: 1.3724 - accuracy: 0.3411
Epoch 2/100
27/27 [==============================] - 0s 1ms/step - loss: 1.2215 - accuracy: 0.4722
Epoch 3/100
27/27 [==============================] - 0s 1ms/step - loss: 1.1572 - accuracy: 0.5325
Epoch 4/100
27/27 [==============================] - 0s 1ms/step - loss: 1.1256 - accuracy: 0.5487
Epoch 5/100
27/27 [==============================] - 0s 1ms/step - loss: 1.1069 - accuracy: 0.5510
Epoch 6/100
27/27 [==============================] - 0s 1ms/step - loss: 1.0943 - accuracy: 0.5452
Epoch 7/100
27/27 [==============================] - 0s 1ms/step - loss: 1.0845 - accuracy: 0.5580
Epoch 8/100
27/27 [==============================] - 0s 2ms/step - loss: 1.0771 - accuracy: 0.5580
Epoch 9/100
27/27 [==============================] - 0s 1ms/step - loss: 1.0704 - accuracy: 0.5580
Epoch 10/100
27/27 [==============================] - 0s 1ms/step - loss: 1.0654 - accuracy: 0.5557
Epoch 11/

In [ ]:
model1.evaluate( test_input, test_output )

7/7 [==============================] - 0s 4ms/step - loss: 0.9768 - accuracy: 0.6157


[0.9768213629722595, 0.6157407164573669]